In [2]:
import pandas as pd
from collections import defaultdict

def process_inventory(transaction_path, product_map_path, output_path):
    # Load and merge data
    transactions = pd.read_csv(transaction_path)
    product_map = pd.read_csv(product_map_path).set_index('product_id')['product_name']
    
    # Filter only investment transactions and relevant columns
    investments = transactions[
        (transactions['TRANSACTION_TYPE'] == 'INVESTMENT') & 
        (transactions['TYPE'].isin(['BUY', 'SELL']))
    ][['T_DATE', 'PRODUCT_ID', 'TYPE', 'UNIT']].copy()
    
    # Convert dates to datetime and sort
    investments['T_DATE'] = pd.to_datetime(investments['T_DATE'])
    investments = investments.sort_values('T_DATE')
    
    # Initialize tracking structures
    inventory = defaultdict(int)
    history = []
    current_quantities = {}

    for idx, row in investments.iterrows():
        product_id = row['PRODUCT_ID']
        action = row['TYPE']
        units = row['UNIT']
        
        # Update inventory
        if action == 'BUY':
            inventory[product_id] += units
        elif action == 'SELL':
            inventory[product_id] = max(0, inventory[product_id] - units)
        
        # Get product name
        product_name = product_map.get(product_id, f'Unknown Product {product_id}')
        
        # Record history
        history.append({
            'Date': row['T_DATE'].strftime('%Y-%m-%d'),
            'Product_ID': product_id,
            'Product_Name': product_name,
            'Transaction_Type': action,
            'Units': units,
            'Cumulative_Quantity': inventory[product_id]
        })
        
        # Track current quantity for potential optimization
        current_quantities[product_id] = inventory[product_id]

    # Create DataFrame and merge with product names
    result = pd.DataFrame(history)
    
    # Add product name to results
    result['Product_Name'] = result['Product_ID'].map(product_map)
    
    # Handle unknown products
    result['Product_Name'] = result['Product_Name'].fillna(result['Product_ID'].apply(
        lambda x: f'Unknown Product {x}'))
    
    # Select final columns
    final_output = result[[
        'Date', 'Product_ID', 'Product_Name', 
        'Transaction_Type', 'Units', 'Cumulative_Quantity'
    ]]
    
    # Save to CSV
    final_output.to_csv(output_path, index=False)
    print(f"Successfully exported inventory history to {output_path}")
    return final_output

# Example usage
if __name__ == "__main__":
    transaction_file = "transaction_path.csv"
    product_map_file = "product_id_to_name.csv"
    output_file = "inventory_history.csv"
    
    inventory_history = process_inventory(transaction_file, product_map_file, output_file)


Successfully exported inventory history to inventory_history.csv


In [3]:
import pandas as pd

# Load files
product_df = pd.read_csv("product_id_to_name.csv")
txn_df = pd.read_csv("transaction_path.csv")

# Filter only investment transactions
txn = txn_df[txn_df['TRANSACTION_TYPE'] == 'INVESTMENT'].copy()

# Only keep relevant columns
txn = txn[['T_DATE', 'PRODUCT_ID', 'TYPE', 'UNIT']]

# Convert date to datetime for sorting and processing
txn['T_DATE'] = pd.to_datetime(txn['T_DATE'], errors='coerce')

# Sort by product and date
txn = txn.sort_values(['PRODUCT_ID', 'T_DATE'])

# Prepare mapping from product_id to name
product_map = dict(zip(product_df['product_id'], product_df['product_name']))

# Track cumulative quantity for each product
records = []
cumulative = {}

for row in txn.itertuples(index=False):
    date, pid, typ, unit = row
    if pd.isnull(pid) or pd.isnull(unit):  # skip bad rows
        continue
    pid = int(pid)
    unit = float(unit)
    prev = cumulative.get(pid, 0)
    if typ == 'BUY':
        new = prev + unit
    elif typ == 'SELL':
        new = prev - unit
    else:
        continue
    cumulative[pid] = new
    records.append({
        'Product_ID': pid,
        'Product_Name': product_map.get(pid, f'Unknown Product {pid}'),
        'Date': date.strftime('%Y-%m-%d'),
        'Transaction_Type': typ,
        'Units': unit,
        'Cumulative_Quantity': new
    })

# Create DataFrame and sort by Product_Name, then Date
result = pd.DataFrame(records)
result = result.sort_values(['Product_Name', 'Date']).reset_index(drop=True)

# Save to CSV
result.to_csv("inventory_history_by_product.csv", index=False)
print("Exported inventory_history_by_product.csv")

# Preview
print(result.head(20))


Exported inventory_history_by_product.csv
    Product_ID                  Product_Name        Date Transaction_Type  \
0      1009100                         3M Co  2021-06-29              BUY   
1      1009100                         3M Co  2025-03-26             SELL   
2      1046315                58.COM INC-ADR  2018-02-28              BUY   
3      1046315                58.COM INC-ADR  2020-04-30             SELL   
4      1046198  AAC TECHNOLOGIES HOLDINGS IN  2017-02-28              BUY   
5      1046198  AAC TECHNOLOGIES HOLDINGS IN  2017-05-02             SELL   
6      1050536   ACADIA Pharmaceuticals Inc   2025-01-21              BUY   
7      1050536   ACADIA Pharmaceuticals Inc   2025-01-21              BUY   
8      1046394                      ACER INC  2018-01-02              BUY   
9      1046394                      ACER INC  2018-01-02              BUY   
10     1046394                      ACER INC  2018-05-02              BUY   
11     1046394                    

In [5]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Load files
product_map = pd.read_csv("product_id_to_name.csv")
dividends = pd.read_csv("new_ticker_dividends.csv")

# Create a clean name matching dictionary
product_names = product_map['product_name'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
clean_names = {name: orig for name, orig in zip(product_names, product_map['product_name'])}

def standardize_name(row):
    # Skip exact matches
    if row['Name'] in product_map['product_name'].values:
        return row['Name']
    
    # Fuzzy match with preprocessing
    clean_input = row['Name'].lower().replace(',', '').replace('.', '')
    match, score = process.extractOne(clean_input, 
                                    clean_names.keys(),
                                    scorer=fuzz.token_sort_ratio)
    
    # Validate match quality
    if score >= 90:  # High confidence threshold
        return clean_names[match]
    elif score >= 60:  # Medium confidence - flag for review
        print(f"Review: '{row['Name']}' → '{clean_names[match]}' ({score}%)")
        return f"{clean_names[match]} (NEEDS REVIEW)"
    else:
        return row['Name']

# Apply standardization
dividends['Standardized_Name'] = dividends.apply(standardize_name, axis=1)

# Generate reports
exact_matches = dividends[dividends['Name'] == dividends['Standardized_Name']]
potential_issues = dividends[dividends['Standardized_Name'].str.contains("NEEDS REVIEW")]
unmatched = dividends[dividends['Standardized_Name'] == dividends['Name']]

print(f"Exact matches: {len(exact_matches)}/{len(dividends)}")
print(f"Potential issues needing review: {len(potential_issues)}")
print(f"Unmatched names: {len(unmatched)}")

# Export results
dividends.to_csv("standardized_dividends.csv", index=False)
potential_issues.to_csv("needs_review.csv", index=False)
unmatched.to_csv("unmatched_names.csv", index=False)


Review: 'POSCO Holdings' → 'VIPSHOP HOLDINGS' (73%)
Review: 'Kakao' → 'KAKAO CORP' (67%)
Review: 'LGELECTRONICS' → 'LG ELECTRONICS INC' (71%)
Review: 'NAVER' → 'NAVER CORP' (67%)
Review: 'SK hynix' → 'SK HYNIX INC' (80%)
Review: 'DAEWOONG PHARM' → 'Daewoong Pharmaceutical Co Ltd' (64%)
Review: 'KIA CORP.' → 'KT CORP' (80%)
Review: 'HyundaiMtr' → 'HYUNDAI MOTOR' (87%)
Review: 'LOTTE SHOPPING' → 'LOTTE SHOPPING LTD' (88%)
Review: 'PETROCHINA' → 'PetroChina Co. Ltd.' (74%)
Review: 'MTR CORPORATION' → 'S-1 CORPORATION' (83%)
Review: 'HSBC HOLDINGS' → 'HSBC Holdings PLC' (87%)
Review: 'KINGSOFT' → 'KINGSOFT CORP LTD' (64%)
Review: 'CHINA OILFIELD' → 'CHINA OILFIELD SERVICES-H' (74%)
Review: 'PING AN' → 'Amgen Inc' (62%)
Review: 'SWIRE PACIFIC A' → 'SWIRE PACIFIC LTD - CL A' (81%)
Review: 'NAN YA PLASTIC' → 'NAN YA PLASTICS CORP.' (82%)
Review: 'FORMOSA PLASTIC' → 'FORMOSA PLASTICS CORP' (83%)
Review: 'CHINA RES LAND' → 'CHINA RESOURCES LAND LTD' (74%)
Review: 'UNITED MICRO ELECTRONICS' → 'L

In [11]:
import pandas as pd
import csv
from datetime import datetime
import re

def load_data():
    # Load datasets with proper CSV handling
    transactions = pd.read_csv(
        "transaction_path.csv",
        parse_dates=['T_DATE'],
        quoting=csv.QUOTE_ALL,
        engine='python',
    )
    products = pd.read_csv("product_id_to_name.csv")
    dividends = pd.read_csv("standardized_dividends.csv")
    
    # Clean portfolio_id column
    transactions['PORTFOLIO_ID'] = pd.to_numeric(
        transactions['PORTFOLIO_ID'].str.extract('(\d+)')[0],
        errors='coerce'
    ).fillna(0).astype(int)
    
    # Create mappings
    id_to_name = dict(zip(products['product_id'], products['product_name']))
    name_to_id = {v: k for k, v in id_to_name.items()}
    
    return transactions, dividends, id_to_name, name_to_id

def process_inventory(transactions):
    # Filter and prepare transactions
    inv_transactions = transactions[
        (transactions['TRANSACTION_TYPE'].isin(['INVESTMENT', 'STOCK_SPLIT'])) &
        (transactions['TYPE'].isin(['BUY', 'SELL']))
    ].copy()
    
    # Convert relevant columns
    inv_transactions['PRODUCT_ID'] = pd.to_numeric(
        inv_transactions['PRODUCT_ID'], errors='coerce'
    ).fillna(0).astype(int)
    
    inv_transactions['UNIT'] = pd.to_numeric(
        inv_transactions['UNIT'], errors='coerce'
    ).fillna(0)
    
    # Sort by date and product
    inv_transactions = inv_transactions.sort_values(['PRODUCT_ID', 'PORTFOLIO_ID', 'T_DATE'])
    
    return inv_transactions

def handle_stock_splits(inv_transactions):
    # Process stock splits using safe regex parsing
    splits = inv_transactions[inv_transactions['TRANSACTION_TYPE'] == 'STOCK_SPLIT']
    split_adjustments = defaultdict(float)
    
    for _, row in splits.iterrows():
        if pd.isna(row['NOTES']):
            continue
            
        # Extract split ratio using regex
        match = re.search(r'@(\d+):(\d+)', row['NOTES'])
        if match:
            _, new = map(int, match.groups())
            split_adjustments[row['PRODUCT_ID']] = new
        else:
            print(f"Warning: Invalid split format in transaction {row['TRANSACTION_ID']}")
    
    return split_adjustments

def calculate_inventory(inv_transactions, split_adjustments):
    # Track inventory with split adjustments
    inventory = defaultdict(lambda: defaultdict(float))
    history = []

    for _, row in inv_transactions.iterrows():
        product_id = row['PRODUCT_ID']
        portfolio = row['PORTFOLIO_ID']
        units = row['UNIT']
        action = row['TYPE']
        date = row['T_DATE']
        
        # Apply stock split adjustments
        if split_ratio := split_adjustments.get(product_id):
            inventory[product_id][portfolio] *= split_ratio
            split_adjustments.pop(product_id)  # Prevent reapplying
        
        # Update inventory
        if action == 'BUY':
            inventory[product_id][portfolio] += units
        elif action == 'SELL':
            inventory[product_id][portfolio] = max(0, inventory[product_id][portfolio] - units)
        
        history.append({
            'Date': date.date(),
            'Product_ID': product_id,
            'Portfolio_ID': portfolio,
            'Transaction_Type': action,
            'Units': units,
            'Cumulative_Quantity': inventory[product_id][portfolio]
        })
    
    return pd.DataFrame(history)

def calculate_dividends(holdings_df, dividends_df, name_to_id):
    dividend_payments = []
    
    # Convert holdings date to datetime
    holdings_df['Date'] = pd.to_datetime(holdings_df['Date'])
    
    for _, div_row in dividends_df.iterrows():
        product_name = div_row['Standardized_Name']
        if (product_id := name_to_id.get(product_name)) is None:
            continue
            
        # Process all dividend columns
        for i in range(1, 7):
            date_col = f'Dividend_{i}_Date'
            amount_col = f'Dividend_{i}_Amount'
            
            if pd.notna(div_row[date_col]) and pd.notna(div_row[amount_col]):
                div_date = pd.to_datetime(div_row[date_col])  # Keep as datetime64
                div_amount = div_row[amount_col]
                
                # Filter holdings with proper datetime comparison
                holdings = holdings_df[
                    (holdings_df['Product_ID'] == product_id) &
                    (holdings_df['Date'] <= div_date)
                ]
                
                if not holdings.empty:
                    latest_holding = holdings.sort_values('Date').iloc[-1]
                    payment = latest_holding['Cumulative_Quantity'] * div_amount
                    
                    dividend_payments.append({
                        'Product_ID': product_id,
                        'Product_Name': product_name,
                        'Dividend_Date': div_date.strftime('%Y-%m-%d'),  # Format for output
                        'Shares_Held': latest_holding['Cumulative_Quantity'],
                        'Dividend_per_Share': div_amount,
                        'Total_Payment': payment
                    })
    
    return pd.DataFrame(dividend_payments)

def main():
    # Load and process data
    transactions, dividends, id_to_name, name_to_id = load_data()
    inv_transactions = process_inventory(transactions)
    
    # Handle stock splits
    split_adjustments = handle_stock_splits(inv_transactions)
    
    # Calculate inventory history
    inventory_history = calculate_inventory(inv_transactions, split_adjustments)
    
    # Calculate dividend payments
    dividend_results = calculate_dividends(inventory_history, dividends, name_to_id)
    
    # Save results
    inventory_history.to_csv('inventory_history_detailed.csv', index=False)
    dividend_results.to_csv('dividend_payments_complete.csv', index=False)
    print("Processing complete. Results saved to CSV files.")

if __name__ == "__main__":
    main()


<>:19: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_37158/1696431897.py:19: SyntaxWarning: invalid escape sequence '\d'
  transactions['PORTFOLIO_ID'].str.extract('(\d+)')[0],


Processing complete. Results saved to CSV files.
